In [32]:
# Importing Required libraries
import json
import networkx as nx

In [37]:
# building the directed graph for imports
# using generated report

with open("black/src/output.json", "r") as f:
    data = json.load(f)
G = nx.DiGraph()

for module, info in data.items():
    if not "black" in module:
        continue
    G.add_node(module)
    if not "imports" in info:
        continue
    for dep in info["imports"]:
        if not "black" in dep: # Not interested in external module imports
            continue
        G.add_edge(module, dep)

In [43]:
# 1. identifying highly coupled modules
def find_highly_coupled_modules(graph, top_n=5):
    in_degrees = dict(graph.in_degree())
    sorted_in_degrees = sorted(in_degrees.items(), key=lambda x: x[1], reverse=True)
    return sorted_in_degrees[:top_n]

print("Highly Coupled Modules:")
for mod, deg in find_highly_coupled_modules(G):
    print(f" - {mod} (imported by {deg} modules)")


Highly Coupled Modules:
 - black (imported by 18 modules)
 - black.mode (imported by 11 modules)
 - black.nodes (imported by 9 modules)
 - black.output (imported by 7 modules)
 - black.report (imported by 4 modules)


In [44]:
# 1.b Identifying modules that import the highest number of other modules
def find_modules_with_most_imports(graph, top_n=5):
    out_degrees = dict(graph.out_degree())
    sorted_out_degrees = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)
    return sorted_out_degrees[:top_n]

print("Modules that Import the Most Other Modules:")
for mod, deg in find_modules_with_most_imports(G):
    print(f" - {mod} (imports {deg} modules)")


Modules that Import the Most Other Modules:
 - black (imports 15 modules)
 - black.linegen (imports 9 modules)
 - black.trans (imports 7 modules)
 - black.concurrency (imports 5 modules)
 - black.files (imports 5 modules)


In [39]:
# 2. Detect cyclic dependencies
def detect_cycles(graph):
    return list(nx.simple_cycles(graph))

print("Cyclic Dependencies:")
cycles = detect_cycles(G)
if not cycles:
    print(" - No cycles detected.")
else:
    for c in cycles:
        print(" - Cycle:", " -> ".join(c))

Cyclic Dependencies:
 - Cycle: black
 - Cycle: black.comments -> black.nodes -> black.strings -> black -> black.linegen -> black.trans
 - Cycle: black.comments -> black.nodes -> black.strings -> black -> black.linegen
 - Cycle: black.comments -> black.nodes -> black.strings -> black
 - Cycle: black.comments -> black.nodes -> black.mode -> black -> black.linegen -> black.trans
 - Cycle: black.comments -> black.nodes -> black.mode -> black -> black.linegen
 - Cycle: black.comments -> black.nodes -> black.mode -> black
 - Cycle: black.comments -> black.nodes -> black.cache -> black.mode -> black -> black.linegen -> black.trans
 - Cycle: black.comments -> black.nodes -> black.cache -> black.mode -> black -> black.linegen
 - Cycle: black.comments -> black.nodes -> black.cache -> black.mode -> black
 - Cycle: black.comments -> black.nodes -> black.cache -> black -> black.linegen -> black.trans
 - Cycle: black.comments -> black.nodes -> black.cache -> black -> black.linegen
 - Cycle: black.co

In [40]:

# 3. Check for unused and disconnected modules
def find_disconnected_modules(graph):
    return [node for node in graph.nodes if graph.in_degree(node) == 0 and graph.out_degree(node) == 0]

print("Disconnected Modules:")
for mod in find_disconnected_modules(G):
    print(f" - {mod}")

Disconnected Modules:
 - black.resources
 - black.schema


In [50]:
# 4. Assess depth of dependencies
def assess_dependency_depth(graph):
    depth_map = {}
    for node in graph.nodes:
        try:
            length = max(len(path) for target in graph.nodes if nx.has_path(graph, node, target) for path in nx.all_simple_paths(graph, node, target))
        except ValueError:
            length = 0
        depth_map[node] = length
    return sorted(depth_map.items(), key=lambda x: x[1], reverse=True)

print("Dependency Depth (Top 5):")
i = 1
for mod, depth in assess_dependency_depth(G)[:5]:
    print(f"{i} - {mod}: depth {depth}")
    i+=1

Dependency Depth (Top 5):
1 - black.files: depth 12
2 - black.linegen: depth 12
3 - black.concurrency: depth 11
4 - black.handle_ipynb_magics: depth 11
5 - black.debug: depth 11


In [49]:
# 1. Dependency impact assessment
def impact_of_module_change(graph, module):
    if module not in graph:
        return []
    # Modules that depend on this module (reverse dependencies)
    return list(nx.descendants(graph.reverse(copy=False), module))

core_module = "black.linegen"  

print(f"\n5. Impact of Changes in Core Module '{core_module}':")
i = 1
for mod in impact_of_module_change(G, core_module):
    print(f"{i} - {mod}")
    i+=1


5. Impact of Changes in Core Module 'black.linegen':
1 - black.lines
2 - black.handle_ipynb_magics
3 - black.cache
4 - black.concurrency
5 - black.mode
6 - black.nodes
7 - black.debug
8 - black.ranges
9 - black.files
10 - black.brackets
11 - black.parsing
12 - black.report
13 - black.strings
14 - black.trans
15 - black
16 - black.__main__
17 - black.comments


In [46]:
# 2. Risk Assesssment
def risk_assessment(graph):
    # Modules that, if changed, affect many others
    risk_scores = {mod: len(impact_of_module_change(graph, mod)) for mod in graph.nodes}
    return sorted(risk_scores.items(), key=lambda x: x[1], reverse=True)

print("\n6. Modules That Put System at Risk if Modified (Top 5):")
for mod, score in risk_assessment(G)[:5]:
    print(f" - {mod} (affects {score} modules)")


6. Modules That Put System at Risk if Modified (Top 5):
 - black.const (affects 18 modules)
 - black.output (affects 18 modules)
 - black._width_table (affects 18 modules)
 - black.numerics (affects 18 modules)
 - black.rusty (affects 18 modules)
